In [4]:
import pymongo as pg

client=pg.MongoClient("mongodb://localhost:27017")
db=client["Scraping"]
my_collection=db["movies"]


In [5]:
cursor = db.my_collection.find({"title":"Emad"})
for i in cursor:
    print(i)

1) Quel est le film le plus long ?

In [6]:
film_plus_longue=my_collection.find().sort("runtime",pg.DESCENDING).limit(1)[0]
film_plus_longue
print("le film le plus long est {} avec une durée de {} minutes." .format(film_plus_longue["title"],film_plus_longue["runtime"]))

le film le plus long est Gone with the Wind avec une durée de 238 minutes.


2) Quels sont les 5 films les mieux notés ?

In [7]:
film_mieux_note=my_collection.find().sort("rating_score",pg.DESCENDING).limit(5)
df_film_mieux_note=pd.DataFrame()
for film in film_mieux_note:
    new_rows=pd.DataFrame({'title':[film['title']],'score':[film['rating_score']]})
    df_film_mieux_note=pd.concat([df_film_mieux_note,new_rows],ignore_index=True)
df_film_mieux_note

,title,score
0,The Shawshank Redemption,9.3
1,The Godfather,9.2
2,12 Angry Men,9.0
3,The Lord of the Rings: The Return of the King,9.0
4,Schindler's List,9.0


3) Dans combien de films a joué Morgan Freeman ? Tom Cruise ?

In [8]:
filter={'casting_principal':'Morgan Freeman'}
nombre_film_Morgan_Freeman=my_collection.count_documents(filter)
print( "le nomrbre de filme de Morgan Freeman est {}  " .format(nombre_film_Morgan_Freeman))


le nomrbre de filme de Morgan Freeman est 7  


In [9]:

filter={'casting_principal':'Tom Cruise'}
nombre_film_Tom_cruise=my_collection.count_documents(filter)
print( "le nomrbre de filme de Tom cruise est {}  " .format(nombre_film_Tom_cruise))


le nomrbre de filme de Tom cruise est 1  


4) Quels sont les 3 meilleurs films d’horreur ? Dramatique ? Comique ?


In [10]:
trois_meilleur_film_horreur=my_collection.find({'genre':'Horror'}).sort('rating_score',pg.DESCENDING).limit(3)
for film in trois_meilleur_film_horreur :
    print('le film de horreur {} a un score {}' .format(film['title'],film['rating_score']) )

le film de horreur Psycho a un score 8.5
le film de horreur Alien a un score 8.5
le film de horreur The Shining a un score 8.4


In [11]:
trois_meilleur_film_Dramatique=my_collection.find({'genre':'Drama'}).sort('rating_score',pg.DESCENDING).limit(3)
for film in trois_meilleur_film_Dramatique :
    print('le film dramatique {} a un score {}' .format(film['title'],film['rating_score']) )

le film dramatique The Shawshank Redemption a un score 9.3
le film dramatique The Godfather a un score 9.2
le film dramatique Schindler's List a un score 9.0


In [12]:
trois_meilleur_film_comique=my_collection.find({'genre':'Comedy'}).sort('rating_score',pg.DESCENDING).limit(3)
for film in trois_meilleur_film_comique :
    print('le film de comique {} a un score {}' .format(film['title'],film['rating_score']) )

le film de comique Life Is Beautiful a un score 8.6
le film de comique The Intouchables a un score 8.5
le film de comique Modern Times a un score 8.5


5) Parmi les 100 films les mieux notés, quel pourcentage sont américains ? Français ?

6) Quel est la durée moyenne d’un film en fonction du genre ?


In [13]:
pipline=[{ "$group":{"_id": "$genre","avgDuration":{"$avg":"$runtime"}}}]

result=my_collection.aggregate(pipline)
result
# for group in result:
#     print(group)
df_avrage_dure=pd.DataFrame()
for film in result:
    new_rows=pd.DataFrame({'genre':[film['_id']],'avgDuration':[round(film['avgDuration'],2)]})
    df_avrage_dure=pd.concat([df_avrage_dure,new_rows],ignore_index=True)
df_avrage_dure

,genre,avgDuration
0,"[Mystery, Thriller]",121.00
1,[Comedy],94.00
2,"[Crime, Drama, Thriller]",124.20
3,"[Comedy, Drama, Romance]",103.75
4,"[Action, Adventure, Mystery]",136.00
...,...,...
99,"[Comedy, Romance]",113.50
100,"[Drama, Western]",147.50
101,"[Drama, Sport]",126.00
102,"[Adventure, Comedy, Drama]",95.00


In [14]:
query={}
query["casting_principal"] = {"$elemMatch":{"$in":[ "Morgan Freeman", "Tim Robbins"]}}
result = my_collection.find(query)
for i in result :
    print(i)

{'_id': ObjectId('639c3124eee4a28db9bdfa14'), 'rating_score': 9.3, 'title': 'The Shawshank Redemption', 'titre_original': None, 'years': '1994', 'public': 'R', 'summary': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.', 'genre': ['Drama'], 'runtime': 142, 'casting_principal': ['Tim Robbins', 'Morgan Freeman', 'Bob Gunton', 'William Sadler', 'Clancy Brown', 'Gil Bellows', 'Mark Rolston', 'James Whitmore', 'Jeffrey DeMunn', 'Larry Brandenburg', 'Neil Giuntoli', 'Brian Libby', 'David Proval', 'Joseph Ragno', 'Jude Ciccolella', 'Paul McCrane', 'Renee Blaine', 'Scott Mann']}
{'_id': ObjectId('639c312eeee4a28db9bdfa1a'), 'rating_score': 8.6, 'title': 'Se7en', 'titre_original': None, 'years': '1995', 'public': 'R', 'summary': 'Two detectives, a rookie and a veteran, hunt a serial killer who uses the seven deadly sins as his motives.', 'genre': ['Crime', 'Drama', 'Mystery'], 'runtime': 127, 'casting_principal': ['Morgan F